In [ ]:
#Read Data From CSV file
import pandas as pd
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.ones(ratings.shape) * 99 #24983*100 and initzalize all be unknown 99 and ratings.shape[0] return 24983
    train = ratings.copy()
    #Make all unknown numbers 99 be 0
    train = train - np.ones(ratings.shape) * 99
    for user in xrange(ratings.shape[0]):
        #Randomly pick 10 ratings from each user to be in test case (include unknown ratings)
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size = 20, replace = False)
        #Set chosen be zero in training
        train [user, test_ratings] = 0.
        #Assign chosen rating to test case
        test[user, test_ratings] = ratings[user, test_ratings]
    
    train = train + np.ones(ratings.shape) * 99
    #Make sure train and test are disjoint 
    #assert(np.all((train * test) == 0))
    return train, test

In [ ]:
#Call the function and then assign train and test
train, test = train_test_generation(ratings)
trainMAE = train.copy()
#Masked for CF and not masked for SVD
train = np.ma.masked_where(train == 99, train)
test = np.ma.masked_where(test == 99, test)

In [ ]:
import json
data = {}
data['userID'] = 0
data['businessID'] = 0
data['rating'] = 0
with open('file.txt', 'w') as file:
    file.write(json.dumps(save))
    file.write('\n')

for indexI in range(24983):
    for indexJ in range(100):
        test = data.copy()
        test['userID'] = indexI
        test['businessID'] = indexJ
        test['rating'] = ratings[indexI][indexJ]
        with open('file.txt', 'a') as file:
            file.write(json.dumps(test))
            file.write('\n')

In [ ]:
with open("file.txt", "r") as ins:
    array = []
    for line in ins:
        array.append(line)

In [ ]:
import ast
a = []
for indexI in range(2498300):
    temp = ast.literal_eval(array[indexI])
    a.append(temp)

In [ ]:
#Compute MAE in training and test
def get_mae(pred, actual):
    mae_sum = 0
    count = 0
    for indexI in range(24983):
        for indexJ in range(100):
            if actual[indexI][indexJ] != 99:
                mae_sum = mae_sum + np.abs(pred[indexI][indexJ] - actual[indexI][indexJ])
                count = count + 1    
    mae_sum = float(mae_sum) / count
    return mae_sum

In [ ]:
from collections import defaultdict
import random
###initialzing 
User_perBusiness = defaultdict(list)
Business_perUser = defaultdict(list) 
UserBusiness_rating = {} 
User_bias = {} 
Business_bias = {} 
GammaUser = {}
GammaBusiness = {}

alpha = 0 
for l in a[:2498300]: 
    user,business = l['userID'],l['businessID'] 
    #Each Business - Multi users
    User_perBusiness[business].append(l['userID']) 
    #Each User - Multi Businesses
    Business_perUser[user].append(l['businessID']) 
    #Each User visits business rating
    UserBusiness_rating[user+business] = l['rating'] 
    User_bias[user] = 0 
    Business_bias[business] = 0 
    GammaUser[user] = random.uniform(0, 1)
    GammaBusiness[business] = random.uniform(0, 1)
    Prev_MSE = 0 #randomly initializing previous MSE 
    Cur_MSE = 10 #randomly intitializing current MSE 
    lam = 5
    #lam=5 best now;
###############################################################################

###############################################################################
###Iterative procedure in fixing GammaUser
while abs(Prev_MSE - Cur_MSE) > 0.000001: 
    Prev_MSE = Cur_MSE 
    #Alternating Least Squares
    
    ## update business Gamma 
    for i in GammaBusiness: 
        temp1 = sum([(UserBusiness_rating[u+i] - (alpha + User_bias[u] + Business_bias[i])) * GammaUser[u] for u in User_perBusiness[i]])
        temp2 = sum([GammaUser[u] * GammaUser[u] for u in User_perBusiness[i]])
        GammaBusiness[i] = temp1 / (lam + temp2)
    
    ## update alpha 
    alpha = [l['rating'] - (User_bias[l['userID']] + Business_bias[l['businessID']]) - (GammaUser[l['userID']] * GammaBusiness[l['businessID']]) for l in a[:2498300]] 
    alpha = sum(alpha) / 2498300 
        
    ## update user bias 
    for u in User_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + Business_bias[i]) - (GammaUser[u] * GammaBusiness[i])  for i in Business_perUser[u]]) 
        User_bias[u] = temp / (lam + len(Business_perUser[u]))
        
    ## update business bias 
    for i in Business_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + User_bias[u]) - (GammaUser[u] * GammaBusiness[i]) for u in User_perBusiness[i]]) 
        Business_bias[i] = temp / (lam + len(User_perBusiness[i]))
        
    ## compute MSE  and f=alpha+BetaU+BetaI+GammaU*GammaI
    diff = [(l['rating'] - (alpha + User_bias[l['userID']] + Business_bias[l['businessID']] + GammaUser[l['userID']] * GammaBusiness[l['businessID']]))**2 for l in a[:2498300]] 
    
    ##test on regularizer
    regularizer = 0
    for u in User_bias:
        regularizer += User_bias[u]**2
    for i in Business_bias:
        regularizer += Business_bias[i]**2
    for u in GammaUser:
        regularizer += GammaUser[u]**2
    for i in GammaBusiness:
        regularizer += GammaBusiness[i]**2
    Cur_MSE = sum(diff) / len(diff) + lam * regularizer
    
###############################################################################
    
###############################################################################    
###Iterative procedure in fixing GammaBusiness
Prev_MSE = 0 #randomly initializing previous MSE 
Cur_MSE = 10 #randomly intitializing current MSE 
while abs(Prev_MSE - Cur_MSE) > 0.000001: 
    Prev_MSE = Cur_MSE 
    #Alternating Least Squares
    
    ## update user Gamma 
    for u in GammaUser: 
        temp1 = sum([(UserBusiness_rating[u+i] - (alpha + User_bias[u] + Business_bias[i])) * GammaBusiness[i] for i in Business_perUser[u]])
        temp2 = sum([GammaBusiness[i] * GammaBusiness[i] for i in Business_perUser[u]])
        GammaUser[u] = temp1 / (lam + temp2)
       
    ## update alpha 
    alpha = [l['rating'] - (User_bias[l['userID']] + Business_bias[l['businessID']]) - (GammaUser[l['userID']] * GammaBusiness[l['businessID']]) for l in a[:2498300]] 
    alpha = sum(alpha) / 2498300 
        
    ## update user bias 
    for u in User_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + Business_bias[i]) - (GammaUser[u] * GammaBusiness[i])  for i in Business_perUser[u]]) 
        User_bias[u] = temp / (lam + len(Business_perUser[u]))
        
    ## update business bias 
    for i in Business_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + User_bias[u]) - (GammaUser[u] * GammaBusiness[i]) for u in User_perBusiness[i]]) 
        Business_bias[i] = temp / (lam + len(User_perBusiness[i]))
        
    ## compute MSE  and f=alpha+BetaU+BetaI+GammaU*GammaI
    diff = [(l['rating'] - (alpha + User_bias[l['userID']] + Business_bias[l['businessID']] + GammaUser[l['userID']] * GammaBusiness[l['businessID']]))**2 for l in a[:2498300]] 
    
    ##test on regularizer
    regularizer = 0
    for u in User_bias:
        regularizer += User_bias[u]**2
    for i in Business_bias:
        regularizer += Business_bias[i]**2
    for u in GammaUser:
        regularizer += GammaUser[u]**2
    for i in GammaBusiness:
        regularizer += GammaBusiness[i]**2
    Cur_MSE = sum(diff) / len(diff) + lam * regularizer
    
############################################################################### 

###############################################################################   
###Iterative procedure 
Prev_MSE = 0 #randomly initializing previous MSE 
Cur_MSE = 10 #randomly intitializing current MSE 
while abs(Prev_MSE - Cur_MSE) > 0.000000001: 
    Prev_MSE = Cur_MSE 
    #Alternating Least Squares    
    
    ## update alpha 
    alpha = [l['rating'] - (User_bias[l['userID']] + Business_bias[l['businessID']]) - (GammaUser[l['userID']] * GammaBusiness[l['businessID']]) for l in a[:2498300]] 
    alpha = sum(alpha) / 2498300 
        
    ## update user bias 
    for u in User_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + Business_bias[i]) - (GammaUser[u] * GammaBusiness[i])  for i in Business_perUser[u]]) 
        User_bias[u] = temp / (lam + len(Business_perUser[u]))
        
    ## update business bias 
    for i in Business_bias: 
        temp = sum([UserBusiness_rating[u+i] - (alpha + User_bias[u]) - (GammaUser[u] * GammaBusiness[i]) for u in User_perBusiness[i]]) 
        Business_bias[i] = temp / (lam + len(User_perBusiness[i]))
        
    ## compute MSE  and f=alpha+BetaU+BetaI+GammaU*GammaI
    diff = [(l['rating'] - (alpha + User_bias[l['userID']] + Business_bias[l['businessID']] + GammaUser[l['userID']] * GammaBusiness[l['businessID']]))**2 for l in a[:2498300]] 
    
    ##test on regularizer
    regularizer = 0
    for u in User_bias:
        regularizer += User_bias[u]**2
    for i in Business_bias:
        regularizer += Business_bias[i]**2
    for u in GammaUser:
        regularizer += GammaUser[u]**2
    for i in GammaBusiness:
        regularizer += GammaBusiness[i]**2
    Cur_MSE = sum(diff) / len(diff) + lam * regularizer

In [ ]:
PredictionRating = [] 
for u in User_bias:
    for i in Business_bias: 
        prediction_ALS_temp = alpha + User_bias[u] + Business_bias[i] + GammaUser[u] * GammaBusiness[i]
        PredictionRating.append(prediction_ALS_temp)
count = 0
for indexI in range(24983):
    for indexJ in range(100):
        prediction_ALS[indexI][indexJ] = PredictionRating[count]
        count = count + 1

In [ ]:
print 'ALS Train MAE: ' + str(get_mae(prediction_ALS, train))
print 'ALS Test MAE: ' + str(get_mae(prediction_ALS, test))